In [30]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
import subprocess
#import pyjq as jq

#from goog_api import google_api_key
#import googlemaps
#gmaps = googlemaps.Client(key=google_api_key)

In [31]:
# import data
#glob_name = "yelp_academic_dataset_" + local_dat_name + '.json' # business, checkin, user, review, tip
def glob_name(name):
    return "yelp_academic_dataset_" + name + ".json"

business_data = pd.read_json(os.path.join(os.getcwd(), 'data', glob_name('business')), lines=True)
#review_data = pd.read_json(os.path.join(os.getcwd(), 'data', glob_name('review')), lines=True)
#df = pd.Dataframe.from_dict()
df = pd.DataFrame.from_dict(business_data)

In [32]:
# filter by state, and by food.
user_state = 'CA'

df = df.loc[df['state'] == user_state]
df = df.loc[df['categories'].str.contains("food|restaurant", na=False, flags=re.IGNORECASE, regex=True)]

In [33]:
# filter categories by the user's food preference
#df.head()
user_prefs = ['Vegan']
user_prefs = "|".join(user_prefs)
df = df.loc[df['categories'].str.contains(user_prefs, na=False, flags=re.IGNORECASE, regex=True)]
df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
921,vLT1KtrA9bWvjFOg-0xVIg,Pieology Pizzeria,"7000 Hollister Ave, Ste F",Goleta,CA,93117,34.430453,-119.872345,3.5,131,0,"{'RestaurantsAttire': ''casual'', 'Alcohol': '...","Restaurants, Gluten-Free, Pizza, Vegan, Fast Food","{'Monday': '0:0-0:0', 'Tuesday': '10:0-20:0', ..."
6300,iHdrLu8deF5GceB0V1rPhA,Oliver's,1198 Coast Village Rd,Santa Barbara,CA,93108,34.421334,-119.644981,4.5,313,1,"{'RestaurantsAttire': 'u'dressy'', 'BikeParkin...","American (Traditional), Vegan, Restaurants","{'Tuesday': '17:0-21:0', 'Wednesday': '17:0-21..."
18784,Js3m_GdqNUGnEgXJ8WMtfQ,Hana Kitchen,503 State St,Santa Barbara,CA,93101,34.416504,-119.695618,3.0,253,1,"{'RestaurantsDelivery': 'True', 'DogsAllowed':...","Bubble Tea, Restaurants, Food, Asian Fusion, V...","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."
19072,GBD-YjcOD1nG0wv4L0Lvfg,Venus in Furs,18 E Cota St,Santa Barbara,CA,93101,34.418217,-119.696120,5.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","American (Traditional), Restaurants, Wine Bars...","{'Wednesday': '16:0-22:0', 'Thursday': '16:0-2..."
20444,Ksz_jsyyK3Zjlq_Cz81ZBA,Adama Vegan Comfort Cuisine,428 Chapala St,Santa Barbara,CA,93101,34.415688,-119.696007,4.0,238,0,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Restaurants, Vegetarian, Vegan","{'Monday': '17:0-21:0', 'Tuesday': '17:0-21:0'..."


In [34]:
# small utility that writes out all possible categories to a txt file.
# used by chatting system.
x = list(df['categories'].str.split(', '))
categories = []
for lst in x:
    for i in lst:
        if i not in categories:
            categories.append(i)
with open('categories.txt', 'w') as f:
    f.write('\n'.join(categories))

In [35]:
# drop the unused columns
new_df = df[['business_id','name','latitude','longitude','categories','stars', 'review_count']].copy()

In [22]:
# create a new rating column based on the baye's prob of stars and review count.
def set_bayes(R, W):
    def bayesian_probability(rating, review_count):
        return (W*R + review_count * rating) / (W + review_count)
    return bayesian_probability

bayes_prob = set_bayes(2, 3)

new_df['new_rating'] = df.apply(lambda row: bayes_prob(row['stars'], row['review_count']), axis=1)
new_df.sort_values('new_rating', ascending=False)
new_df.to_json('latlon.json',orient="records", lines=True)

In [36]:
top_five = new_df.head()['business_id']



for i in top_five:
    filt = f".[] | select(.business_id == \'{i}\')"
    filtered_res = jq.all(filt,review_dat)
    print(filtered_res)

FileNotFoundError: [Errno 2] No such file or directory: 'yelp_academic_dataset_reviews.json'

In [24]:
jq.all?

Signature:     
jq.all(
    program,
    value=<jq._EmptyValue object at 0x7f5b13f04b60>,
    text=<jq._EmptyValue object at 0x7f5b13f04b60>,
)
Call signature: jq.all(*args, **kwargs)
Type:           cython_function_or_method
String form:    <cyfunction all at 0x7f5a7177f370>
Docstring:      <no docstring>